<a href="https://colab.research.google.com/github/jaeseongrhythm/circadian-predict-AI/blob/main/%EA%B0%80%EC%83%81_%EC%9D%B8%EB%AC%BC_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%83%9D%EC%84%B1%EA%B8%B0_(LCO_%EB%AA%A8%EB%8D%B8_%ED%86%B5%ED%95%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""
가상 인물 데이터 생성기 (LCO 모델 통합 및 사용자 요청 수정)

이 스크립트는 현실적인 가상 인물 프로필을 기반으로 생체 데이터를 시뮬레이션하며,
생성된 광(lux) 및 수면 데이터에 기반하여 LCO(Limit Cycle Oscillator) 모델을 사용한
일주기 리듬 상태 변수를 계산하고 결과에 포함합니다.

주요 변경 사항 (사용자 요청 반영):
- 칼로리 기록 방식 수정: 식사 시간 동안 1분 단위로 칼로리가 분배되어 기록됩니다.
- 결측치 처리 로직 추가: 데이터 생성 후 수치 데이터는 0으로, 종류 데이터는 'none'으로 결측치를 채웁니다.

기존 주요 기능:
- LCO 모델 추가: 일주기 생체리듬 예측 AI의 핵심 물리 모델을 이식하여, 생성된 데이터에 맞는 LCO 상태 변수(x, xc, n)를 계산.
- LCO 계산 로직: 예측 모델의 복잡한 보정 로직 없이, 매일 자정을 기준으로 이전 날의 최종 상태를 이어받아 다음 날의 궤도를 시뮬레이션.
- 시각화 강화: 최종 리포트에 LCO 상태 변수(x, xc) 그래프를 추가하여 자극과 생체리듬의 관계를 시각적으로 분석할 수 있도록 함.
- 라이브러리 추가: LCO 계산을 위해 scipy, tqdm 라이브러리 추가.
"""

import pandas as pd
import numpy as np
import os
import warnings
import math
import matplotlib
# --- 그래프가 화면에 표시되지 않도록 백엔드 설정 ---
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import random
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from tqdm import tqdm


# --- 경고 메시지 무시 ---
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


# --- 기본 설정 ---
# 생성할 데이터 기간 (일)
DATA_DURATION_DAYS = 20
# 하루의 분 단위
DAY_MINUTES = 24 * 60

# =============================================================================
# 1. 가상 인물 프로필 생성 함수
# =============================================================================
def generate_person_profile():
    """
    보고서에 기반하여, 변수 간 상관관계를 고려한 현실적인 개인 프로필을 생성합니다.
    """
    profile = {}

    # 1단계: 핵심 기반 변수 결정 (나이)
    profile['age'] = random.randint(20, 70)

    # 2단계: 주요 특성 변수 결정 (체력 수준, 크로노타입)
    # 나이에 따라 체력 수준 결정 (나이가 들수록 high 확률 감소)
    if profile['age'] < 40:
        fitness_probs = [0.2, 0.5, 0.3]  # low, medium, high
    elif profile['age'] < 60:
        fitness_probs = [0.4, 0.5, 0.1]
    else:
        fitness_probs = [0.6, 0.3, 0.1]
    profile['fitness_level'] = np.random.choice(['low', 'medium', 'high'], p=fitness_probs)

    # 나이에 따라 크로노타입 결정 (나이가 들수록 저녁형 확률 감소)
    if profile['age'] < 30:
        chrono_probs = [0.2, 0.4, 0.4]  # morning, neutral, evening
    elif profile['age'] < 50:
        chrono_probs = [0.4, 0.4, 0.2]
    else:
        chrono_probs = [0.7, 0.2, 0.1]
    profile['chronotype'] = np.random.choice(['morning', 'neutral', 'evening'], p=chrono_probs)

    # 3단계: 파생 변수 결정 (기본 생체 지표)
    fitness_score = {'low': -1, 'medium': 0, 'high': 1}[profile['fitness_level']]
    profile['base_hr'] = round(70 - (fitness_score * 8) + random.uniform(-3, 3))
    profile['base_hrv'] = round(80 - (profile['age'] * 0.8) + (fitness_score * 15) + random.uniform(-5, 5))
    profile['base_respiration_rate'] = round(16 - fitness_score + random.uniform(-1, 1), 1)
    profile['base_skin_temp'] = round(33.5 + random.uniform(-0.5, 0.5), 2)

    # 4단계: 파생 변수 결정 (행동 패턴)
    profile['exercise_freq_prob'] = {'low': 0.15, 'medium': 0.4, 'high': 0.7}[profile['fitness_level']]
    profile['sleep_duration_mean'] = round(7.5 - (profile['age'] - 20) * 0.02 + random.uniform(-0.5, 0.5), 1)

    if profile['chronotype'] == 'morning':
        wake_up_base = 6.5
    elif profile['chronotype'] == 'neutral':
        wake_up_base = 7.5
    else: # evening
        wake_up_base = 8.0
    profile['wake_up_weekday_mean'] = wake_up_base
    profile['wake_up_weekday_std'] = 0.75

    if profile['chronotype'] == 'evening':
        profile['wake_up_weekend_mean'] = wake_up_base + 1.5
    else:
        profile['wake_up_weekend_mean'] = wake_up_base + 1.0
    profile['wake_up_weekend_std'] = 1.5

    return profile


# =============================================================================
# 2. 헬퍼 함수 및 데이터 생성 함수
# =============================================================================
def moving_average_np(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def calculate_sleep_quality(daily_events, person_profile):
    quality_score = 1.0
    age_penalty = max(0, (person_profile['age'] - 30) * 0.005)
    quality_score -= age_penalty
    if daily_events['alcohol_g'].sum() > 0: quality_score -= 0.35
    caffeine_events = daily_events[daily_events['caffeine_g'] > 0]
    if not caffeine_events.empty and caffeine_events.index.hour.max() >= 15: quality_score -= 0.15
    exercise_events = daily_events[daily_events['exercise_event'] > 0]
    if not exercise_events.empty:
        last_exercise_time = exercise_events.index.max()
        if last_exercise_time.hour >= 21 and daily_events.loc[last_exercise_time]['exercise_intensity'] == 'high': quality_score -= 0.1
    meal_events = daily_events[daily_events['meal_event'] > 0]
    if not meal_events.empty and meal_events.index.hour.max() >= 21: quality_score -= 0.1
    smartphone_events = daily_events[daily_events['smartphone_event'] > 0]
    sleep_start_time_series = daily_events[daily_events['is_sleeping'] == 1].index
    if not smartphone_events.empty and not sleep_start_time_series.empty:
        sleep_start_time = sleep_start_time_series.min()
        if (sleep_start_time - smartphone_events.index.max()).total_seconds() / 3600 < 1:
            quality_score -= 0.1
    if daily_events['stress_event'].sum() > 0: quality_score -= 0.15
    if not exercise_events.empty and exercise_events.index.hour.max() < 20: quality_score += 0.05
    return np.clip(quality_score, 0.3, 1.0)

# =============================================================================
# 2.5. 일주기 리듬 (LCO) 모델 함수 (신규 추가)
# =============================================================================
PARAMS = {
    'mu': 0.13, 'q': 1/3, 'k': 0.55, 'alpha0': 0.1, 'I0': 9500,
    'p': 0.5, 'beta': 0.007, 'G': 37, 'rho': 0.032, 'tau_x': 24.2,
}

def _sigmoid(x, k=2, x0=0):
    """Sigmoid helper function."""
    return 1 / (1 + np.exp(-k * (x - x0)))

def lco_model_ode(t, y, params, light_func, sleep_func):
    """
    LCO(Limit Cycle Oscillator) 모델의 미분 방정식.
    St. Hilaire et al., 2007 논문을 기반으로 한 모델을 사용합니다.
    t: 시간 (단위: 시간)
    y: 상태 변수 [x, xc, n]
    """
    x, xc, n = y
    if not np.all(np.isfinite(y)): return [0,0,0]

    mu, q, k, alpha0, I0, p, beta, G, rho, tau_x = params.values()
    I, sigma = light_func(t), sleep_func(t)
    I = max(I, 0)
    alpha = alpha0 * ((I / I0)**p) * (I / (I + 100.0)) if I > 0 else 0
    B_hat = G * (1 - n) * alpha
    B = B_hat * (1 - 0.4 * x) * (1 - 0.4 * xc)

    # 수면 압력(Ns) 계산 로직
    cbt_min_phase_angle = -170.7 * np.pi / 180.0
    current_phase = np.arctan2(xc, x)
    phase_diff_rad = (current_phase - cbt_min_phase_angle + np.pi) % (2 * np.pi) - np.pi
    psi_c_x = phase_diff_rad * (tau_x / (2 * np.pi)) + (tau_x / 2)
    weight_enter = _sigmoid(psi_c_x, k=2, x0=16.5)
    weight_exit = 1 - _sigmoid(psi_c_x, k=2, x0=21.0)
    wmz_weight = weight_enter * weight_exit * sigma
    Ns_hat_normal = rho * (1/3.0 - sigma)
    Ns_hat_wmz = rho * (1/3.0)
    Ns_hat = Ns_hat_normal * (1 - wmz_weight) + Ns_hat_wmz * wmz_weight
    Ns = Ns_hat * (1 - np.tanh(10 * x))

    # 미분 방정식
    dxdt = (np.pi / 12.0) * (xc + mu * (x/3.0 + (4.0/3.0)*x**3 - (256.0/105.0)*x**7) + B + Ns)
    tau_term_sq = (24.0 / (0.99729 * tau_x))**2
    dxc_dt = (np.pi / 12.0) * (q * B * xc - x * (tau_term_sq + k * B))
    dn_dt = 60.0 * (alpha * (1 - n) - beta * n)
    return [dxdt, dxc_dt, dn_dt]

def calculate_lco_trajectory(df, params):
    """
    생성된 데이터(lux, is_sleeping)를 기반으로 LCO 궤도를 하루 단위로 계산합니다.
    """
    print("--- LCO 궤도 계산 시작 ---")
    total_minutes = len(df)
    num_days = total_minutes // DAY_MINUTES
    lco_trajectory = np.zeros((total_minutes, 3))
    y0 = [1.0, 0.0, 0.5]  # 첫 날의 초기 상태

    t_eval_day_hours = np.arange(DAY_MINUTES) / 60.0

    for day in tqdm(range(num_days), desc="LCO 궤도 계산 중"):
        day_start_idx = day * DAY_MINUTES
        day_end_idx = (day + 1) * DAY_MINUTES
        day_df = df.iloc[day_start_idx:day_end_idx]

        # 안정적인 계산을 위해 입력 데이터를 부드럽게 만듭니다.
        lux_smoothed = day_df['lux'].rolling(window=15, min_periods=1, center=True).mean().values
        sleep_smoothed = day_df['is_sleeping'].rolling(window=15, min_periods=1, center=True).mean().values

        light_func = interp1d(t_eval_day_hours, lux_smoothed, kind='linear', fill_value="extrapolate")
        sleep_func = interp1d(t_eval_day_hours, sleep_smoothed, kind='linear', fill_value="extrapolate")

        sol_day = solve_ivp(
            fun=lco_model_ode,
            t_span=[0, 24],
            y0=y0,
            method='BDF',
            args=(params, light_func, sleep_func),
            dense_output=True,
            t_eval=t_eval_day_hours,
            rtol=1e-5,
            atol=1e-8
        )

        if sol_day.success and sol_day.y.shape[1] == DAY_MINUTES:
            lco_trajectory[day_start_idx:day_end_idx, :] = sol_day.y.T
            y0 = sol_day.y[:, -1]  # 현재 날의 마지막 상태를 다음 날의 초기 상태로 사용
        else:
            print(f"경고: Day {day+1} LCO 시뮬레이션 실패. 이전 궤도를 복사합니다.")
            if day > 0:
                lco_trajectory[day_start_idx:day_end_idx, :] = lco_trajectory[(day-1)*DAY_MINUTES:day*DAY_MINUTES, :]
            # y0는 이전 상태를 유지

    print("--- LCO 궤도 계산 완료 ---")
    return lco_trajectory

# =============================================================================
# 3. 생체 데이터 생성 메인 함수
# =============================================================================
def generate_realistic_biometric_data(duration_days, person_profile):
    total_minutes = duration_days * DAY_MINUTES
    timestamps = pd.to_datetime('2025-07-01') + pd.to_timedelta(np.arange(total_minutes), 'm')
    df = pd.DataFrame(index=timestamps)

    # --- 이벤트 스케줄링을 위한 타임라인 생성 ---
    # 0: Awake, 1: Sleep, 2: Meal, 3: Exercise, 4: Stress, 5: Smartphone
    activity_timeline = pd.Series(0, index=timestamps)

    hr_max = 220 - person_profile['age']
    recovery_factor = {'low': 1.2, 'medium': 1.0, 'high': 0.8}[person_profile['fitness_level']]
    # 나이 페널티를 계산하되, 최대값을 15로 제한합니다.
    age_penalty = (person_profile['age'] - 30) * 0.5
    capped_penalty = min(age_penalty, 15) # 최대 감소폭을 15ms로 제한
    age_adjusted_hrv = person_profile['base_hrv'] - capped_penalty

    event_cols = [
        'is_sleeping', 'meal_event', 'exercise_event', 'stress_event', 'smartphone_event',
        'meal_type', 'meal_calories', 'caffeine_g', 'alcohol_g',
        'exercise_type', 'exercise_intensity', 'sleep_quality'
    ]
    for col in event_cols:
        if 'type' in col or 'intensity' in col: df[col] = ''
        else: df[col] = 0.0

    df['lux'] = 0.0
    df['light_color_temp'] = 0.0

    # --- 이벤트 생성 (우선순위: 수면 > 운동 > 식사 > 스트레스 > 스마트폰) ---
    for day in range(duration_days):
        day_start_idx = day * DAY_MINUTES
        current_day_ts = timestamps[day_start_idx]
        is_weekend = current_day_ts.dayofweek >= 5

        # 1. 수면 이벤트 생성
        wake_up_hour = np.random.normal(person_profile[f'wake_up_{"weekend" if is_weekend else "weekday"}_mean'], person_profile[f'wake_up_{"weekend" if is_weekend else "weekday"}_std'])
        sleep_duration = np.random.normal(person_profile['sleep_duration_mean'], 1.0 if is_weekend else 0.5)
        wake_up_minute = int(np.clip(wake_up_hour * 60, 0, DAY_MINUTES - 1))
        sleep_end_abs = day_start_idx + wake_up_minute
        sleep_start_abs = sleep_end_abs - int(sleep_duration * 60)
        effective_sleep_start = max(0, sleep_start_abs)
        effective_sleep_end = min(total_minutes, sleep_end_abs)
        sleep_indices = range(effective_sleep_start, effective_sleep_end)
        if sleep_indices:
            df.iloc[sleep_indices, df.columns.get_loc('is_sleeping')] = 1
            activity_timeline.iloc[sleep_indices] = 1 # 타임라인에 수면 기록

    for day in range(duration_days):
        day_start_idx = day * DAY_MINUTES
        is_weekend = timestamps[day_start_idx].dayofweek >= 5
        # 기상 시간은 수면 블록에서 역산
        day_sleep_indices = df.iloc[day_start_idx : day_start_idx + DAY_MINUTES].index[df.iloc[day_start_idx : day_start_idx + DAY_MINUTES]['is_sleeping'] == 0]
        wake_up_minute = day_sleep_indices[0].hour * 60 + day_sleep_indices[0].minute if not day_sleep_indices.empty else int(person_profile[f'wake_up_{"weekend" if is_weekend else "weekday"}_mean'] * 60)

        # 2. 운동 이벤트 생성
        if np.random.rand() < person_profile['exercise_freq_prob']:
            event_start = day_start_idx + int(np.random.normal(18.5, 1.0) * 60)
            duration = np.random.randint(45, 90)
            event_indices = range(event_start, min(event_start + duration, total_minutes))
            if len(event_indices) > 0 and (activity_timeline.iloc[event_indices] == 0).all():
                activity_timeline.iloc[event_indices] = 3
                df.iloc[event_indices, df.columns.get_loc('exercise_event')] = 1
                exercise_type = np.random.choice(['aerobic', 'anaerobic'])
                exercise_intensity = np.random.choice(['low', 'medium', 'high'])
                df.iloc[event_indices, df.columns.get_loc('exercise_type')] = exercise_type
                df.iloc[event_indices, df.columns.get_loc('exercise_intensity')] = exercise_intensity

        # 3. 식사 이벤트 생성 (수정됨)
        meal_times = [wake_up_minute + np.random.randint(90, 180), int(np.random.normal(19, 1.0) * 60)] if is_weekend else [wake_up_minute + np.random.randint(30, 60), int(np.random.normal(12.5, 0.5) * 60), int(np.random.normal(19.5, 0.75) * 60)]
        for meal_time in meal_times:
            event_start_idx = day_start_idx + meal_time
            duration = np.random.randint(20, 41)  # 식사 시간 20-40분
            total_calories = np.random.randint(400, 1200)
            calories_per_minute = total_calories / duration

            event_indices = range(event_start_idx, min(event_start_idx + duration, total_minutes))
            if len(event_indices) > 0 and (activity_timeline.iloc[event_indices] == 0).all():
                activity_timeline.iloc[event_indices] = 2 # 타임라인에 식사 기록
                df.iloc[event_indices, df.columns.get_loc('meal_event')] = 1
                meal_type = np.random.choice(['balanced', 'high_protein', 'high_fat', 'high_carb'])
                df.iloc[event_indices, df.columns.get_loc('meal_type')] = meal_type
                df.iloc[event_indices, df.columns.get_loc('meal_calories')] = calories_per_minute

                # 카페인/알코올은 식사 시작 시점에만 기록
                if meal_time < 15 * 60 and np.random.rand() < 0.6:
                    df.iloc[event_start_idx, df.columns.get_loc('caffeine_g')] = np.random.uniform(0.08, 0.15)
                if meal_time > 18 * 60 and np.random.rand() < 0.3:
                    df.iloc[event_start_idx, df.columns.get_loc('alcohol_g')] = np.random.uniform(14, 28)

        # 4. 스트레스 이벤트
        if not is_weekend and np.random.rand() < 0.15:
            event_start = day_start_idx + int(np.random.uniform(10, 16) * 60)
            duration = np.random.randint(30, 120)
            event_indices = range(event_start, min(event_start + duration, total_minutes))
            if len(event_indices) > 0 and (activity_timeline.iloc[event_indices] == 0).all():
                activity_timeline.iloc[event_indices] = 4
                df.iloc[event_indices, df.columns.get_loc('stress_event')] = 1

        # 5. 스마트폰 이벤트
        for hour in range(24):
            prob = 0.1
            if hour == wake_up_minute // 60: prob = 0.8
            elif 12 <= hour <= 13: prob = 0.5
            elif 19 <= hour <= 22: prob = 0.6
            if np.random.rand() < prob:
                event_start = day_start_idx + hour * 60 + np.random.randint(0, 50)
                duration = np.random.randint(5, 15)
                event_indices = range(event_start, min(event_start + duration, total_minutes))
                if len(event_indices) > 0 and (activity_timeline.iloc[event_indices] == 0).all():
                    activity_timeline.iloc[event_indices] = 5
                    df.iloc[event_indices, df.columns.get_loc('smartphone_event')] = 1

    # --- 수면의 질 계산 (연속된 수면 블록 기준) ---
    sleep_onsets = df.index[df['is_sleeping'].diff() == 1]
    for sleep_start_time in sleep_onsets:
        sleep_bout_df = df.loc[sleep_start_time:]
        sleep_end_time_candidates = sleep_bout_df.index[sleep_bout_df['is_sleeping'].diff() == -1]
        sleep_end_time = sleep_end_time_candidates[0] if not sleep_end_time_candidates.empty else df.index[-1]
        current_sleep_indices = df.index[(df.index >= sleep_start_time) & (df.index < sleep_end_time)]
        events_before_sleep_start_idx = max(0, df.index.get_loc(sleep_start_time) - DAY_MINUTES)
        events_before_sleep_end_idx = df.index.get_loc(sleep_start_time)
        daily_events_df = df.iloc[events_before_sleep_start_idx:events_before_sleep_end_idx]
        quality = calculate_sleep_quality(daily_events_df, person_profile) if not daily_events_df.empty else 1.0
        df.loc[current_sleep_indices, 'sleep_quality'] = quality

    # --- 생체 데이터 생성 ---
    minute_of_day_series_full = np.tile(np.arange(DAY_MINUTES), duration_days)
    natural_lux = 30000 * np.exp(-0.5 * ((minute_of_day_series_full - 12.5 * 60) / (3.5 * 60))**2)
    natural_color_temp = 3000 + 3500 * np.exp(-0.5 * ((minute_of_day_series_full - 13 * 60) / (4 * 60))**2)

    # KeyError 수정을 위해 컬럼 추가
    df['natural_lux'] = natural_lux
    df['natural_color_temp'] = natural_color_temp

    df['lux'], df['light_color_temp'] = 600.0, 4500.0
    df.loc[(df.index.hour >= 19) & (df.index.hour < 23), ['lux', 'light_color_temp']] = [200.0, 3000.0]
    df.loc[df.index.hour >= 23, ['lux', 'light_color_temp']] = [75.0, 2700.0]
    df.loc[df['is_sleeping'] == 1, ['lux', 'light_color_temp']] = [np.random.uniform(1, 5), 2700.0]

    is_outdoor = (df['exercise_type'] == 'aerobic')
    lunch_outdoor_prob = np.random.rand(len(df)) < 0.2
    is_lunch_outdoor = (df.index.hour >= 12) & (df.index.hour < 14) & lunch_outdoor_prob
    is_outdoor |= is_lunch_outdoor
    df.loc[is_outdoor, 'lux'] = df.loc[is_outdoor, 'natural_lux']
    df.loc[is_outdoor, 'light_color_temp'] = df.loc[is_outdoor, 'natural_color_temp']
    df.loc[df['smartphone_event'] == 1, 'lux'] += 500.0
    df.loc[df['smartphone_event'] == 1, 'light_color_temp'] = 6500.0
    df['lux'] = df['lux'].clip(lower=1.0)

    minute_of_day_series = df.index.minute + df.index.hour * 60
    is_night_blue_light = (df['is_sleeping'] == 0) & (df['light_color_temp'] > 5000) & ((df.index.hour > 21) | (df.index.hour < 4))

    df['heart_rate'] = person_profile['base_hr'] + (5 * np.cos(2 * np.pi * (minute_of_day_series - 14*60) / DAY_MINUTES)) + (-10 * df['is_sleeping'] * df['sleep_quality']) + (2 * is_night_blue_light)
    df['hrv'] = age_adjusted_hrv + (15 * np.cos(2 * np.pi * (minute_of_day_series - 4*60) / DAY_MINUTES)) + (25 * df['is_sleeping'] * df['sleep_quality']) + (-8 * is_night_blue_light)
    df['ambient_temp'] = 22 + 5 * np.sin(2 * np.pi * (minute_of_day_series - 15*60) / DAY_MINUTES)
    df['skin_temp'] = person_profile['base_skin_temp'] + ((df['ambient_temp'] - 22) * 0.15) + (1.5 * np.sin(2 * np.pi * (minute_of_day_series - 17*60) / DAY_MINUTES)) + (-0.5 * is_night_blue_light)
    df['respiration_rate'] = person_profile['base_respiration_rate'] + (1.5 * np.cos(2 * np.pi * (minute_of_day_series - 14*60) / DAY_MINUTES)) + (-3 * df['is_sleeping'] * df['sleep_quality']) + (1 * is_night_blue_light)

    exercise_indices = df.index[df['exercise_event'] == 1]
    if not exercise_indices.empty:
        for start_idx in df.index[df['exercise_event'].diff() == 1]:
            if df.loc[start_idx, 'exercise_event'] == 0: continue
            intensity_str = df.loc[start_idx, 'exercise_intensity']

            end_candidates = df.loc[start_idx:, 'exercise_event'].diff() == -1
            end_time = end_candidates.idxmax() if end_candidates.any() else df.index[-1]
            duration = end_time - start_idx
            duration_minutes = duration.total_seconds() / 60
            if duration_minutes == 0: continue

            # FutureWarning 수정을 위해 int 캐스팅 및 freq='min' 사용
            current_exercise_indices = pd.date_range(start=start_idx, periods=int(duration_minutes), freq='min')

            intensity_map = {'low': 0.8, 'medium': 1.2, 'high': 1.8}
            resp_peak = 12 * intensity_map.get(intensity_str, 1.0)
            rise_effect = 1 - np.exp(-(np.arange(int(duration_minutes))) / 5)
            df.loc[current_exercise_indices, 'respiration_rate'] += resp_peak * rise_effect

            recovery_duration = int(90 * recovery_factor)
            recovery_indices = pd.date_range(start=current_exercise_indices[-1] + pd.Timedelta(minutes=1), periods=recovery_duration, freq='min')
            recovery_indices = recovery_indices[recovery_indices < df.index[-1]]
            decay_effect = np.exp(-(np.arange(len(recovery_indices))) / 30)
            df.loc[recovery_indices, 'respiration_rate'] += resp_peak * decay_effect

            skin_temp_intensity_map = {'low': 0.7, 'medium': 1.0, 'high': 1.5}
            skin_temp_increase = 1.0 * skin_temp_intensity_map.get(intensity_str, 1.0)
            df.loc[current_exercise_indices, 'skin_temp'] += skin_temp_increase
            df.loc[recovery_indices, 'skin_temp'] += skin_temp_increase * np.exp(-(np.arange(len(recovery_indices))) / 40)

            hr_intensity_map = {'low': 0.55, 'medium': 0.7, 'high': 0.85}
            hrv_intensity_map = {'low': 0.8, 'medium': 1.0, 'high': 1.2}
            target_hr = hr_max * hr_intensity_map.get(intensity_str, 0.7)
            hrv_dip = -25 * hrv_intensity_map.get(intensity_str, 1.0)
            df.loc[current_exercise_indices, 'heart_rate'] += (target_hr - df.loc[start_idx, 'heart_rate']) * rise_effect
            df.loc[current_exercise_indices, 'hrv'] += hrv_dip
            df.loc[recovery_indices, 'hrv'] += (15 / recovery_factor) * np.exp(-0.5 * ((np.arange(len(recovery_indices)) - 60) / (45 * recovery_factor))**2)

    # --- 식사 효과 적용 (수정됨) ---
    meal_start_indices = df.index[df['meal_event'].diff() == 1]
    if not meal_start_indices.empty and df['meal_event'].iloc[0] == 1:
        meal_start_indices = meal_start_indices.insert(0, df.index[0])

    for start_idx in meal_start_indices:
        end_candidates = df.loc[start_idx:, 'meal_event'].diff() == -1
        end_idx = end_candidates.idxmax() if end_candidates.any() else df.index[-1]
        meal_bout_indices = df.index[(df.index >= start_idx) & (df.index < end_idx)]
        if meal_bout_indices.empty: continue

        total_calories = df.loc[meal_bout_indices, 'meal_calories'].sum()
        if total_calories == 0: continue

        resp_peak = 0.5 * (total_calories / 800)
        if df.loc[start_idx, 'meal_type'] == 'high_protein': resp_peak *= 1.5
        duration = 120
        effect_indices = pd.date_range(start=start_idx, periods=duration, freq='min')
        effect_indices = effect_indices[effect_indices < df.index[-1]]
        effect = np.exp(-0.5 * ((np.arange(len(effect_indices)) - 30) / 45)**2)
        df.loc[effect_indices, 'respiration_rate'] += resp_peak * effect
        df.loc[effect_indices, 'skin_temp'] += (0.3 * (total_calories / 800)) * effect
        df.loc[effect_indices, 'heart_rate'] += ((total_calories / 100)) * effect
        df.loc[effect_indices, 'hrv'] += (-(total_calories / 120)) * effect

    for start_idx in df.index[df['caffeine_g'] > 0]:
        resp_peak = 1.0
        duration = 180
        effect_indices = pd.date_range(start=start_idx, periods=duration, freq='min')
        effect_indices = effect_indices[effect_indices < df.index[-1]]
        effect = np.exp(-0.5 * ((np.arange(len(effect_indices)) - 45) / 60)**2)
        df.loc[effect_indices, 'respiration_rate'] += resp_peak * effect
        df.loc[effect_indices, 'skin_temp'] += -0.2 * effect
        df.loc[start_idx, 'heart_rate'] += 8
        df.loc[effect_indices, 'hrv'] += -10 * effect

    for start_idx in df.index[df['alcohol_g'] > 0]:
        resp_dip = -1.5
        duration = 180
        effect_indices = pd.date_range(start=start_idx, periods=duration, freq='min')
        effect_indices = effect_indices[effect_indices < df.index[-1]]
        effect = np.exp(-0.5 * ((np.arange(len(effect_indices)) - 60) / 70)**2)
        df.loc[effect_indices, 'respiration_rate'] += resp_dip * effect
        df.loc[effect_indices, 'skin_temp'] += 1.2 * np.exp(-0.5 * ((np.arange(len(effect_indices)) - 30) / 50)**2)
        df.loc[start_idx, 'heart_rate'] += 10
        hrv_dip_duration = int(8 * 60)
        hrv_effect_indices = pd.date_range(start=start_idx, periods=hrv_dip_duration, freq='min')
        hrv_effect_indices = hrv_effect_indices[hrv_effect_indices < df.index[-1]]
        df.loc[hrv_effect_indices, 'hrv'] *= 0.6
        sleep_in_window = df.loc[hrv_effect_indices]
        sleep_indices = sleep_in_window.index[sleep_in_window['is_sleeping']==1]
        if not sleep_indices.empty:
            df.loc[sleep_indices, 'heart_rate'] += 5
            df.loc[sleep_indices, 'hrv'] *= 0.8

    stress_indices = df.index[df['stress_event'] == 1]
    if not stress_indices.empty:
        df.loc[stress_indices, 'respiration_rate'] += 8
        df.loc[stress_indices, 'heart_rate'] += 15
        df.loc[stress_indices, 'hrv'] *= 0.7

    for col, std in [('heart_rate', 1.5), ('hrv', 2.5), ('respiration_rate', 0.5), ('skin_temp', 0.1)]:
        df[col] += np.random.normal(0, std, total_minutes)

    df['heart_rate'] = df['heart_rate'].clip(lower=35, upper=hr_max + 5)
    df['hrv'] = df['hrv'].clip(lower=10)
    df['skin_temp'] = df['skin_temp'].clip(lower=30, upper=38)
    df['respiration_rate'] = df['respiration_rate'].clip(lower=8, upper=40)

    # --- 결측치 처리 (신규 추가) ---
    print("--- 결측치 처리 시작 ---")
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            # 수치형 데이터는 0으로 채움
            df[col] = df[col].fillna(0)
        elif pd.api.types.is_object_dtype(df[col]):
            # 종류(문자열) 데이터는 'none'으로 채움
            df[col] = df[col].replace('', 'none').fillna('none')
    print("--- 결측치 처리 완료 ---")

    df.index.name = 'timestamp'
    return df.drop(columns=['natural_lux', 'natural_color_temp'])

# =============================================================================
# 4. 시각화 및 메인 실행 블록
# =============================================================================
def find_all_markers(df_unscaled):
    df = df_unscaled.copy()
    df['corrected_skin_temp'] = df['skin_temp'] - df['ambient_temp']
    all_markers = {'onset': [], 'nadir': [], 'offset': []}
    num_days = len(df) // DAY_MINUTES
    for day in range(num_days):
        day_start, day_end = day * DAY_MINUTES, (day + 1) * DAY_MINUTES
        day_df = df.iloc[day_start:day_end]
        sleep_hr = day_df['heart_rate'][day_df['is_sleeping'] > 0.5]
        if not sleep_hr.empty:
            all_markers['nadir'].append(sleep_hr.idxmin())
        temp_series = day_df['corrected_skin_temp'].values
        if len(temp_series) > 40:
            temp_deriv_smoothed = moving_average_np(np.gradient(moving_average_np(temp_series, 30)), 10)
            offset_to_align = len(temp_series) - len(temp_deriv_smoothed)
            onset_candidates = np.where(temp_deriv_smoothed > 0.001)[0] + offset_to_align
            offset_candidates = np.where(temp_deriv_smoothed < -0.001)[0] + offset_to_align
            if len(onset_candidates) > 0: all_markers['onset'].append(day_df.index[onset_candidates[0]])
            if len(offset_candidates) > 0: all_markers['offset'].append(day_df.index[offset_candidates[-1]])
    return all_markers

def plot_full_data_report(df_raw, all_markers, output_dir, person_profile, profile_id):
    total_days = len(df_raw) // DAY_MINUTES
    days_per_chunk = 7
    num_chunks = math.ceil(total_days / days_per_chunk)
    for i in range(num_chunks):
        start_day = i * days_per_chunk
        end_day = min((i + 1) * days_per_chunk, total_days)
        chunk_df = df_raw.iloc[start_day * DAY_MINUTES : end_day * DAY_MINUTES]
        if chunk_df.empty: continue
        # LCO 그래프 추가를 위해 subplot 개수 및 크기 조정
        fig, axs = plt.subplots(7, 1, figsize=(20, 35), sharex=True)

        profile_info = f"Person #{profile_id} (Age: {person_profile['age']}, Fitness: {person_profile['fitness_level']}, Chronotype: {person_profile['chronotype']})"
        fig.suptitle(f'Biometric Data Report - {profile_info}', fontsize=20, y=0.97)

        # --- Ax 1: 심박수, 피부온도, 수면의 질 ---
        ax1 = axs[0]
        ax1_twin = ax1.twinx()
        ax1.set_title('Heart Rate, Skin Temperature & Sleep Quality', fontsize=14)
        ax1.set_ylabel('Heart Rate (bpm)', color='tab:red')
        p1, = ax1.plot(chunk_df.index, chunk_df['heart_rate'], color='tab:red', label='Heart Rate', zorder=10)
        ax1.tick_params(axis='y', labelcolor='tab:red')
        ax1_twin.set_ylabel('Skin Temp (°C)', color='tab:purple')
        p2, = ax1_twin.plot(chunk_df.index, chunk_df['skin_temp'], color='tab:purple', label='Skin Temp', zorder=10)
        ax1_twin.tick_params(axis='y', labelcolor='tab:purple')
        sleep_fill = ax1.fill_between(chunk_df.index, ax1.get_ylim()[0], ax1.get_ylim()[1], where=chunk_df['is_sleeping'] > 0.5, color='gray', alpha=0.2, label='Sleep', zorder=0)
        ax1_twin2 = ax1.twinx()
        ax1_twin2.spines['right'].set_position(('outward', 60))
        ax1_twin2.set_ylabel('Sleep Quality', color='tab:cyan')
        p3, = ax1_twin2.plot(chunk_df.index, chunk_df['sleep_quality'].where(chunk_df['is_sleeping'] > 0.5), color='tab:cyan', label='Sleep Quality', drawstyle='steps-post', zorder=11)
        ax1_twin2.tick_params(axis='y', labelcolor='tab:cyan')
        ax1_twin2.set_ylim(0, 1)
        ax1.legend(handles=[p1, p2, sleep_fill, p3], loc='upper left')

        # --- Ax 2: HRV, 호흡수, 운동 ---
        ax2 = axs[1]
        ax2_twin = ax2.twinx()
        ax2.set_title('HRV, Respiration Rate & Exercise', fontsize=14)
        ax2.set_ylabel('HRV (ms)', color='tab:green')
        p1, = ax2.plot(chunk_df.index, chunk_df['hrv'], color='tab:green', label='HRV', zorder=10)
        ax2.tick_params(axis='y', labelcolor='tab:green')
        ax2_twin.set_ylabel('Respiration (brpm)', color='tab:brown')
        p2, = ax2_twin.plot(chunk_df.index, chunk_df['respiration_rate'], color='tab:brown', label='Respiration Rate', zorder=10)
        ax2_twin.tick_params(axis='y', labelcolor='tab:brown')
        ex_fill = ax2.fill_between(chunk_df.index, ax2.get_ylim()[0], ax2.get_ylim()[1], where=chunk_df['exercise_event'] > 0.5, color='orange', alpha=0.3, label='Exercise', zorder=0)

        for idx, row in chunk_df[chunk_df['exercise_event'].diff() == 1].iterrows():
             if row['exercise_event'] == 1:
                ax2.text(idx, ax2.get_ylim()[1]*0.9, f"{row['exercise_type']}\n({row['exercise_intensity']})", ha='left', va='top', fontsize=9, color='darkorange')
        ax2.legend(handles=[p1, p2, ex_fill], loc='upper left')

        # --- Ax 3: 광 노출 및 색온도 ---
        ax3 = axs[2]
        ax3_twin = ax3.twinx()
        ax3.set_title('Light Exposure & Color Temperature', fontsize=14)
        ax3.set_ylabel('Lux (log scale)', color='orange')
        ax3.plot(chunk_df.index, chunk_df['lux'], color='orange', label='Lux')
        ax3.set_yscale('log')
        ax3.tick_params(axis='y', labelcolor='orange')
        ax3.set_ylim(bottom=1)
        ax3_twin.set_ylabel('Color Temperature (K)', color='dodgerblue')
        ax3_twin.plot(chunk_df.index, chunk_df['light_color_temp'], color='dodgerblue', label='Color Temp (K)', alpha=0.7)
        ax3_twin.tick_params(axis='y', labelcolor='dodgerblue')
        ax3.legend(loc='upper left')
        ax3_twin.legend(loc='upper right')

        # --- Ax 4: 식사 이벤트 ---
        ax4 = axs[3]
        ax4.set_title('Meal Events', fontsize=14)
        ax4.set_ylabel('Calories (kcal/min)') # Y-축 레이블 변경
        ax4.plot(chunk_df.index, chunk_df['meal_calories'], color='skyblue', label='Calories per Minute', drawstyle='steps-post')
        for idx, row in chunk_df[chunk_df['meal_event'].diff() == 1].iterrows():
            if row['meal_event'] == 1 and row['meal_type'] != 'none':
                ax4.text(idx, row['meal_calories'], f" {row['meal_type']}", ha='left', va='bottom', fontsize=9, rotation=45)
        ax4.legend(loc='upper left')

        # --- Ax 5: 카페인 및 알코올 섭취 ---
        ax5 = axs[4]
        ax5.set_title('Caffeine & Alcohol Intake', fontsize=14)
        p1 = ax5.bar(chunk_df.index, chunk_df['alcohol_g'], width=0.01, color='rebeccapurple', label='Alcohol (g)')
        ax5.set_ylabel('Alcohol (g)', color='rebeccapurple')
        ax5.tick_params(axis='y', labelcolor='rebeccapurple')
        ax5_twin = ax5.twinx()
        p2 = ax5_twin.bar(chunk_df.index, chunk_df['caffeine_g'], width=0.01, color='saddlebrown', label='Caffeine (g)')
        ax5_twin.set_ylabel('Caffeine (g)', color='saddlebrown')
        ax5_twin.tick_params(axis='y', labelcolor='saddlebrown')
        handles = [p1, p2]
        labels = [h.get_label() for h in handles]
        ax5.legend(handles, labels, loc='upper left')

        # --- Ax 6: 원본 이벤트 데이터 ---
        ax6 = axs[5]
        ax6.set_title('Raw Events for Reference', fontsize=14)
        ax6.plot(chunk_df.index, chunk_df['is_sleeping'], label='is_sleeping', drawstyle='steps-post', color='gray')
        ax6.plot(chunk_df.index, chunk_df['meal_event'], label='meal_event', drawstyle='steps-post', color='skyblue')
        ax6.plot(chunk_df.index, chunk_df['exercise_event'], label='exercise_event', drawstyle='steps-post', color='orange')
        ax6.plot(chunk_df.index, chunk_df['stress_event'], label='stress_event', drawstyle='steps-post', color='red')
        ax6.plot(chunk_df.index, chunk_df['smartphone_event'], label='smartphone_event', drawstyle='steps-post', color='blueviolet')
        ax6.set_yticks([0, 1])
        ax6.legend(loc='upper left')

        # --- Ax 7: LCO 모델 상태 변수 (신규 추가) ---
        ax7 = axs[6]
        ax7.set_title('LCO Model State Variables', fontsize=14)
        ax7.plot(chunk_df.index, chunk_df['lco_x'], label='x (Core Body Temp Proxy)', color='darkorange')
        ax7.plot(chunk_df.index, chunk_df['lco_xc'], label='xc (Activity/Wakefulness Proxy)', color='teal')
        ax7.set_ylabel('State Value')
        ax7.legend(loc='upper left')

        # --- 공통 포맷팅 ---
        marker_colors = {'onset': 'green', 'nadir': 'blue', 'offset': 'purple'}
        for ax in axs:
            ax.grid(True, which='major', linestyle='--', linewidth=0.5)
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d\n%H:%M'))
            ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))
            for marker_type, timestamps in all_markers.items():
                for ts in timestamps:
                    if not chunk_df.empty and chunk_df.index[0] <= ts < chunk_df.index[-1]:
                        ax.axvline(x=ts, color=marker_colors[marker_type], linestyle='--', linewidth=1.5, label=f'_{marker_type}')

        from matplotlib.lines import Line2D
        legend_elements = [Line2D([0], [0], color='green', lw=2, linestyle='--', label='Onset (체온 상승)'),
                           Line2D([0], [0], color='blue', lw=2, linestyle='--', label='Nadir (최저 심박수)'),
                           Line2D([0], [0], color='purple', lw=2, linestyle='--', label='Offset (체온 하강)')]
        fig.legend(handles=legend_elements, loc='upper right', fontsize=12)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        save_path = os.path.join(output_dir, f'report_person_{profile_id}.png')
        plt.savefig(save_path)
        plt.close(fig)

def run_data_generation_and_visualization(output_dir, duration_days, person_profile, profile_id):
    """
    데이터 생성, LCO 계산, 시각화, 저장을 총괄하는 메인 함수
    """
    print(f"Generating data for profile #{profile_id}...")

    df_generated = generate_realistic_biometric_data(duration_days, person_profile)

    # --- LCO 궤도 계산 및 데이터프레임에 추가 ---
    lco_traj = calculate_lco_trajectory(df_generated, PARAMS)
    df_generated[['lco_x', 'lco_xc', 'lco_n']] = lco_traj

    all_markers = find_all_markers(df_generated)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    plot_full_data_report(df_generated, all_markers, output_dir, person_profile, profile_id)

    csv_path = os.path.join(output_dir, f'biometric_data_person_{profile_id}.csv')
    df_generated.to_csv(csv_path, index_label='timestamp')

    profile_path = os.path.join(output_dir, f'person_{profile_id}_profile.txt')
    with open(profile_path, 'w', encoding='utf-8') as f:
        f.write(f"--- 가상 인물 프로필 #{profile_id} ---\n")
        for key, value in person_profile.items():
            f.write(f'{key}: {value}\n')

    print(f"Data for profile #{profile_id} saved to '{output_dir}'")


if __name__ == '__main__':
    # --- 생성할 가상 인물 프로필 수 설정 ---
    NUMBER_OF_PROFILES = 1

    print(f"Starting generation for {NUMBER_OF_PROFILES} virtual person profiles.")
    print("="*60)

    for i in range(NUMBER_OF_PROFILES):
        profile_id = i + 1
        person_profile = generate_person_profile()
        output_dir = f"biometric_data_person_{profile_id}"

        run_data_generation_and_visualization(
            output_dir,
            DATA_DURATION_DAYS,
            person_profile,
            profile_id
        )

    print("="*60)
    print(" All tasks completed successfully. ".center(60, "="))
    print("="*60)


Starting generation for 1 virtual person profiles.
Generating data for profile #1...
--- LCO 궤도 계산 시작 ---


LCO 궤도 계산 중: 100%|██████████| 20/20 [00:08<00:00,  2.36it/s]


--- LCO 궤도 계산 완료 ---
Data for profile #1 saved to 'biometric_data_person_1'
============ All tasks completed successfully. =============
